# 1. fewshot template

In [1]:
# 1. 无案例的是时候的回答
import os
import dotenv
from langchain_core.prompts import FewShotPromptTemplate
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

chat_model = ChatOpenAI(model="gpt-4o-mini",
                        temperature=0.4)

res = chat_model.invoke("2 🦜 9是多少?")
print(res.content)

2 🦜 9 这个表达式的意思不太明确。如果你能提供更多的上下文或解释“🦜”的含义，我会很乐意帮助你进行计算或解答。


In [8]:
# 2. 实例的调用
from langchain_core.prompts import PromptTemplate

example_prompt =PromptTemplate.from_template(
  # 注意这里和example里面是对应的
  template="input: {input}\noutput: {output}"
)

examples = [
    {"input": "北京天气怎么样", "output": "北京市"},
    {"input": "南京下雨吗", "output": "南京市"},
    {"input": "武汉热吗", "output": "武汉市"}
]

# 创建few shot prompt
few_shot_template = FewShotPromptTemplate(
  example_prompt = example_prompt,
  examples = examples,
  # 声明在示例后面的提示词模板
  suffix = "input: {input}\noutput:",
  input_variables = ["input"],

)

a_template = few_shot_template.invoke({"input": "上海天气怎么样"})
response = chat_model.invoke(a_template)
print(response.content)

上海市


In [ ]:
# 3. 举例3
#1、创建提示模板
from langchain.prompts import PromptTemplate

# 创建提示模板，配置一个提示模板，将一个示例格式化为字符串
prompt_template = "你是一个数学专家,算式： {input} 值： {output} 使用： {description} "

# 这是一个提示模板，用于设置每个示例的格式
prompt_sample = PromptTemplate.from_template(prompt_template)

#2、提供示例
examples = [
    {"input": "2+2", "output": "4", "description": "加法运算"},
    {"input": "5-2", "output": "3", "description": "减法运算"},
]


#3、创建一个FewShotPromptTemplate对象
from langchain.prompts.few_shot import FewShotPromptTemplate


prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt_sample,
    suffix="你是一个数学专家,算式: {input}  值: {output}",
    input_variables=["input", "output"]
)
# print(prompt.invoke({"input":"2*5", "output":"10"}))

#4、初始化大模型，然后调用
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(model="gpt-4o-mini")

result = chat_model.invoke(prompt.invoke({"input":"2*5", "output":"10"}))
print(result.content)  # 使用: 乘法运算

In [30]:

from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

prompt_template1 = PromptTemplate.from_template(
  template="你是一名数学专家，算式： {input} 值： {output} 使用： {description} ",
)

examples =[
    {"input": "2+2", "output": "4", "description": "加法运算"},
    {"input": "5-2", "output": "3", "description": "减法运算"},
]

few_shot_template1 = FewShotPromptTemplate(
  example_prompt = prompt_template1,
  examples = examples,
  # 这里说明suffix这个地方，输入和input应该是要匹配的，而且格式化的时候，要把input和output都传进去,传多传少都会报错，关键是要看你的format或者invoke需要传入哪些
  suffix = "你是一名数学专家，算式： {input}",
  input_variables = ["input"],
)

# few_shot_template1.invoke({"input": "2*5", "output": "10"})
from langchain_openai import ChatOpenAI
chat_model1 = ChatOpenAI(model="gpt-4o-mini")
result1 = chat_model1.invoke(few_shot_template1.invoke({"input": "2*5"}))
print(result1.content)

算式： 2 * 5  
值： 10  
使用： 乘法运算


# 2. fewshotChatMessageTemplate

In [ ]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)

# 1.示例消息格式
examples = [
    {"input": "1+1等于几？", "output": "1+1等于2"},
    {"input": "法国的首都是？", "output": "巴黎"}
]

# 2.定义示例的消息格式提示词模版
msg_example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

# 3.定义FewShotChatMessagePromptTemplate对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=msg_example_prompt,
    examples=examples
)
# 4.输出格式化后的消息
print(few_shot_prompt.format())

In [32]:
# 1.导入相关包
from langchain_core.prompts import (FewShotChatMessagePromptTemplate, ChatPromptTemplate)

# 2.定义示例组
examples = [
    {"input": "2🦜3", "output": "8"},
    {"input": "2🦜4", "output": "16"},
]

# 3.定义示例的消息格式提示词模版
example_prompt = ChatPromptTemplate.from_messages([
    ('human', '{input} 是多少?'),
    ('ai', '{output}')
])

# 4.定义FewShotChatMessagePromptTemplate对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,  # 示例组
    example_prompt=example_prompt,  # 示例提示词词模版
)
# 5.输出完整提示词的消息模版
final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '你是一个数学奇才'),
        few_shot_prompt,
        ('human', '{input}'),
    ]
)

#6.提供大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

chat_model = ChatOpenAI(model="gpt-4o-mini",
                        temperature=0.4)

chat_model.invoke(final_prompt.invoke(input="2🦜10")).content

'2🦜10 等于 1024。'

# 3. 示例选择器

In [34]:
# 1.导入相关包
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
import os
import dotenv
from langchain_openai import OpenAIEmbeddings

dotenv.load_dotenv()


embeddings_model = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

# 3.定义示例组
examples = [
    {
        "question": "谁活得更久，穆罕默德·阿里还是艾伦·图灵?",
        "answer": """
        接下来还需要问什么问题吗？
        追问：穆罕默德·阿里去世时多大年纪？
        中间答案：穆罕默德·阿里去世时享年74岁。
        """,
    },
    {
        "question": "craigslist的创始人是什么时候出生的？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：谁是craigslist的创始人？
        中级答案：Craigslist是由克雷格·纽马克创立的。
        """,
    },
    {
        "question": "谁是乔治·华盛顿的外祖父？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：谁是乔治·华盛顿的母亲？
        中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿。
        """,
    },
    {
        "question": "《大白鲨》和《皇家赌场》的导演都来自同一个国家吗？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：《大白鲨》的导演是谁？
        中级答案：《大白鲨》的导演是史蒂文·斯皮尔伯格。
        """,
    },
]

# 4.定义示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 这是可供选择的示例列表
    examples,
    # 这是用于生成嵌入的嵌入类，用于衡量语义相似性
    embeddings_model,
    # 这是用于存储嵌入并进行相似性搜索的 VectorStore 类
    Chroma,
    # 这是要生成的示例数量
    k=1,
)

# 选择与输入最相似的示例
question = "玛丽·鲍尔·华盛顿的父亲是谁?"
selected_examples = example_selector.select_examples({"question": question})
print(f"与输入最相似的示例：{selected_examples}")

# for example in selected_examples:
#     print("\n")
#     for k, v in example.items():
#         print(f"{k}: {v}")

与输入最相似的示例：[{'question': '谁是乔治·华盛顿的外祖父？', 'answer': '\n        接下来还需要问什么问题吗？\n        追问：谁是乔治·华盛顿的母亲？\n        中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿。\n        '}]


In [ ]:
# 2. 结合少量示例给大模型
# 1.导入相关包
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

# 2.定义示例提示词模版
example_prompt = PromptTemplate.from_template(
    template="Input: {input}\nOutput: {output}",
)

# 3.创建一个示例提示词模版
examples = [
    {"input": "高兴", "output": "悲伤"},
    {"input": "高", "output": "矮"},
    {"input": "长", "output": "短"},
    {"input": "精力充沛", "output": "无精打采"},
    {"input": "阳光", "output": "阴暗"},
    {"input": "粗糙", "output": "光滑"},
    {"input": "干燥", "output": "潮湿"},
    {"input": "富裕", "output": "贫穷"},
]

# 4.定义嵌入模型
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

# 5.创建语义相似性示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    FAISS,
    k=2,
)
#或者
#example_selector = SemanticSimilarityExampleSelector(
#    examples,
#    embeddings,
#    FAISS,
#    k=2
#)

# 6.定义小样本提示词模版
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个词组的反义词",
    suffix="Input: {word}\nOutput:",
    input_variables=["word"],
)

response = similar_prompt.invoke({"word":"忧郁"})
print(response.text)

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

# 3
example_prompt = PromptTemplate.from_template(
  template = "input: {input}\noutput: {output}"
)
# 2
examples = [
    {"input": "高兴", "output": "悲伤"},
    {"input": "高", "output": "矮"},
    {"input": "长", "output": "短"},
    {"input": "精力充沛", "output": "无精打采"},
    {"input": "阳光", "output": "阴暗"},
    {"input": "粗糙", "output": "光滑"},
    {"input": "干燥", "output": "潮湿"},
    {"input": "富裕", "output": "贫穷"},
]

# 4.定义嵌入模型
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

# 5. 语义选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
  examples,
  embeddings,
  FAISS,
  k=2,
)

# 1
similar_prompt = FewShotPromptTemplate(
  example_selector=example_selector,
  example_prompt=example_prompt,
  prefix="给出每个词组的反义词",
  suffix="Input: {word}\nOutput:",
  input_variables=["word"],
)

from langchain_openai import ChatOpenAI
chat_model = ChatOpenAI(model="gpt-4o-mini")
response = chat_model.invoke(similar_prompt.invoke({"word":"忧郁"}))
print(response.content)

快乐


In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

# 3
example_prompt = PromptTemplate.from_template(
  template = "input: {input}\noutput: {output}"
)
# 2
examples = [
    {"input": "高兴", "output": "悲伤"},
    {"input": "高", "output": "矮"},
    {"input": "长", "output": "短"},
    {"input": "精力充沛", "output": "无精打采"},
    {"input": "阳光", "output": "阴暗"},
    {"input": "粗糙", "output": "光滑"},
    {"input": "干燥", "output": "潮湿"},
    {"input": "富裕", "output": "贫穷"},
]

# 4.定义嵌入模型
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

# 5. 语义选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
  examples,
  embeddings,
  # 我试了如果用Chroma结果会反
  FAISS,
  k=2,
)

# 1
similar_prompt = FewShotPromptTemplate(
  example_selector=example_selector,
  example_prompt=example_prompt,
  prefix="给出每个词组的反义词",
  suffix="Input: {word}\nOutput:",
  input_variables=["word"],
)

# similar_prompt.invoke({"word":"忧郁"})
# print(example_selector)
# for _ in example_selector:
#     print(_)

# 4.定义示例选择器
# example_selector = SemanticSimilarityExampleSelector.from_examples(
#     # 这是可供选择的示例列表
#     examples,
#     # 这是用于生成嵌入的嵌入类，用于衡量语义相似性
#     embeddings_model,
#     # 这是用于存储嵌入并进行相似性搜索的 VectorStore 类
#     Chroma,
#     # 这是要生成的示例数量
#     k=1,
# )

# 选择与输入最相似的示例
input = "开心"
# 输入字典
selected_examples = example_selector.select_examples({"input": input})
print(f"与输入最相似的示例：{selected_examples}")

与输入最相似的示例：[{'input': '高兴', 'output': '悲伤'}, {'input': '精力充沛', 'output': '无精打采'}]
